# Visualization and Conversion of VTI Image Data from HDF5 Files

This notebook is dedicated to processing and visualizing image data sequences stored within HDF5 files. The primary focus is on extracting these data sequences, which are stored as numpy arrays, and then either visualizing them as 2D images or converting them back into 2D slices in VTI format for further analysis and visualization in compatible tools like ParaView.

Written by Monica Rotulo (monica.rotulo@surf.nl)

## Initialization

Let's start installing the required packages and importing the libraries:

In [ ]:
# Install packages 
!pip install vtk
!pip install h5py
!pip install pyvista
!pip install matplotlib

In [ ]:
#!/usr/bin/env python
import os
import re
from typing import List, Tuple
import numpy as np

import pyvista as pv
import h5py

import vtk
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk

We also define some VTK image data (Uniform rectilinear) constants:

In [ ]:
EXTENT_SIZE_3D = (101, 101, 51)  # Total dimensions including the zero index
EXTENT_SIZE_2D = (101, 101)  # Expected dimensions for 2D rendering

ORIGIN = (0, 0, 0)
SPACING = (1, 1, 1)
CELL_DATA = "Spin"  # Used specifically for the Scalars attribute in CellData

## numpy_to_vtk_file Function

Convert a 2D NumPy array to a `vtkImageData` object and save it as a VTK file.

### Parameters:
- `data_array` (`np.ndarray`): The NumPy array to convert.
- `output_path` (`str`): The file path where the VTK file will be saved.
- `spacing` (`tuple`, optional): The spacing between data points in the `vtkImageData`. Defaults to `(1.0, 1.0, 1.0)`.
- `origin` (`tuple`, optional): The origin of the `vtkImageData`. Defaults to `(0.0, 0.0, 0.0)`.

### Returns:
- `None`: The function does not return a value but writes directly to the specified file path.

### Description:
This function takes a 2D numpy array, converts it into a VTK image data object using `vtk.vtkImageData`, and then writes it to a file specified by `output_path`. The VTK image data includes configurable spacing and origin settings to allow for proper scaling and positioning within the VTK environment.


In [ ]:
def numpy_to_vtk_file(data_array: np.ndarray, output_path: str, spacing=SPACING, origin=ORIGIN) -> None:
    dimx, dimy = data_array.shape
    dimx, dimy = dimx + 1, dimy + 1

    # Create a vtkImageData object
    image_data = vtk.vtkImageData()
    image_data.SetDimensions(dimx, dimy, 1) 
    image_data.SetSpacing(spacing)
    image_data.SetOrigin(origin)
    image_data.AllocateScalars(vtk.VTK_INT, 1)

    # Convert the NumPy array to a VTK array
    vtk_data_array = numpy_to_vtk(num_array=data_array.ravel(), deep=True)

    vtk_data_array.SetName(CELL_DATA)
    vtk_data_array.SetNumberOfComponents(1)

    image_data.GetCellData().SetScalars(vtk_data_array)
    image_data.GetPointData().RemoveArray(0)

    writer = vtk.vtkXMLImageDataWriter()
    writer.SetFileName(output_path)
    writer.SetInputData(image_data)
    writer.SetDataModeToAscii()
    writer.EncodeAppendedDataOff()
    writer.SetCompressor(None)

    writer.Write()

## render_2D_from_numpy Function

Render a 2D image from a 2D slice of a NumPy array and save it as an image file using PyVista.

### Parameters:
- `numpy_array` (`np.ndarray`): A 2D NumPy array or a 2D slice of a 3D array. The shape of the array should be (height, width) or (height, width, 1).
- `filename` (`str`, optional): The name of the file where the image will be saved. Defaults to "visual_np.png".

### Returns:
- `None`: The function does not return any value but writes the rendered image directly to the specified file.

### Description:
This function takes a 2D numpy array or a slice from a 3D numpy array and renders it as a 2D image. The rendering uses PyVista, a 3D plotting and mesh analysis toolkit. The rendering process is performed off-screen, and the resulting image is saved to the specified file. This function is ideal for visualizing 2D data slices from 3D datasets.

In [ ]:
def check_array_dimensions(numpy_array: np.ndarray, extent_size: tuple):
    expected_shape = tuple(e - 1 for e in extent_size)
    if numpy_array.shape != expected_shape:
        raise ValueError(
            f"Array dimensions {numpy_array.shape} do not match the expected dimensions {expected_shape}."
        )

    return True


def render_2D_from_numpy(numpy_array: np.ndarray, filename: str = "visual_np.png"):
    check_array_dimensions(numpy_array, extent_size=EXTENT_SIZE_2D)

    grid = pv.ImageData()  
    grid.dimensions = (
        numpy_array.shape[0] + 1,
        numpy_array.shape[1] + 1,
        1,
    )  # Note the ordering of dimensions
    grid.spacing = SPACING
    grid.origin = ORIGIN
    grid.cell_data[CELL_DATA] = numpy_array.flatten(order="C")

    pv.start_xvfb()
    plotter = pv.Plotter(off_screen=True)
    plotter.add_mesh(grid, cmap="viridis", show_edges=False)

    plotter.show(auto_close=False)
    plotter.screenshot(filename)
    plotter.close()


## visualize_all_sequence_from_numpy Function

Renders each frame from a sequence of NumPy arrays as an image and saves them to disk.

### Parameters:
- `sequence` (`List[np.ndarray]`): A list of NumPy arrays, where each array represents image data for a frame.

### Behavior:
- Iterates through each element in the sequence.
- Checks if the element is a valid NumPy array and renders it as an image using the `render_2D_from_numpy` function.
- If an element is not a NumPy array, skips rendering for that index and prints a message indicating the skipped index.

In [ ]:
def visualize_all_sequence_from_numpy(sequence: List[np.ndarray], filename: str = "instance") -> None:
    for i, frame in enumerate(sequence):
        if isinstance(frame, np.ndarray):
            render_2D_from_numpy(frame, filename=f'{filename}_{i}.png')
        else:
            print(f"Skipping index {i}: not a numpy array")

## H5_Handler Class

A utility class for handling operations on HDF5 files, particularly for extracting and managing sequences of images from experiments.

In [ ]:
class H5_Handler:
    def __init__(self, file_path):
        self.file_path = file_path
        self.experiments_length = self.extract_length(file_path)

    def extract_length(self, filename):
        # Use a regular expression to find 'len' followed by an underscore and one or more digits
        match = re.search(r'len_(\d+)', self.file_path)
        if match:
            return int(match.group(1))  # Convert the matched string (digits only) to an integer
        else:
            return None  # Return None if no match is found

    def load_experiment(self, video_idx):
        """
        Read and return a range of images from the HDF5 file.

        video_index (int): Index of the video for which frames are to be loaded (0-indexed).
        return:numpy.ndarray: Array of frames for the specified video.
        """
        start_idx = video_idx * self.experiments_length
        end_idx = start_idx + self.experiments_length

        with h5py.File(self.file_path, "r") as file:
            images = file["images"][start_idx:end_idx]
        return images


    def get_total_frames(self):
        """
        Get the total number of frames (images) in the HDF5 file.

        return: int, the total number of images in the file.
        """
        with h5py.File(self.file_path, "r") as file:
            num_frames = len(file["images"])
        return num_frames

    def get_total_experiments(self):
        """
        Get the total number of experiments (temporal sequence of images) in the HDF5 file.

        return: int, the total number of experiments stored in the file.
        """
        with h5py.File(self.file_path, "r") as file:
            num_frames = len(file["images"])
        
        return num_frames // self.experiments_length


## Executing the functions

This code snippet demonstrates how to initialize an `H5_Handler` object with a specified HDF5 file. This object will be used to manage data extraction and manipulation for experiments stored in HDF5 format.

After initializing the `data_handler`, you can use it to load data, retrieve information about the number of experiments, frames, etc. 

In [ ]:
data_path = "/insert/your/data/path/all_experiments"
experiment = "exp_1_len_90_2D.h5"

data_handler = H5_Handler(os.path.join(data_path, experiment))
data_handler

You can use the initialized `H5_Handler` object to load a sequence of images from a specific experiment within the HDF5 file

In [ ]:
video_idx = 0 # For example, load frames from video 2 (index 1)
sequence_0 = data_handler.load_experiment(video_idx)
len(sequence_0)

You can now visualize the sequence of images stored in a NumPy array and convert each image in the sequence to a VTI (VTK Image Data) file format for further use in visualization and analysis tools that support VTI.

This approach is particularly useful for processing and analyzing sequences of 2D images extracted from larger datasets, such as time-series data from scientific experiments. The visualization step allows for a quick quality check of the images, while converting to VTI files prepares the data for advanced visualization and analysis in applications like ParaView.

### Example Output Files:
- Image files: `image_0.png`, `image_1.png`, ..., `image_n.png`
- VTI files: `process_2d.vti.0`, `process_2d.vti.1`, ..., `process_2d.vti.n`

The saved VTI files can be loaded into VTK-compatible visualization software to explore the data in more detail or to perform computational analyses on the image data.

In [ ]:
visualize_all_sequence_from_numpy(sequence_0, 'image')

for i, frame in enumerate(sequence_0):
  numpy_to_vtk_file(frame, f'process_2d.vti.{i}')